## Dependências

In [1]:
import pandas as pd
import numpy as np
import json

# Libs para realizar o resquest
import requests

# Atenticação na API
from requests.auth import HTTPBasicAuth

# Parse de date
from dateutil.parser import parse

# Expressão regular
import re

# Contador
from collections import Counter

# Barra de progresso
from tqdm import tqdm

# Inteagir com o OS
import os

# Permitir interagir co mdatas
from datetime import timezone, datetime

# Compara se um item de um conjunto não está no outro
from operator import is_not

#Transforma lista de lista, em lista 1d
from nltk import flatten

In [3]:
# URL de acesso ao zendesk para consumir a API
base_url = 'https://empresa.zendesk.com'

# Usuário e senha para consultar a API do zendesk (a senha por ser o API token)
# Esse usuário deve ser Admin, ou ter os privilégios de Admin.
user = 'usuario@dominio.com.br'
passw = 'senha_do_usuario'

## Teste Tickets atualizados

In [5]:
print('Digite a data (DD-MM-AAAA): Todos os Tickets criados depois dessa data e atualizado recentemente')
str_tipo_consulta = input('Data:')

# Preencher data para consulta (Ano, Mes, Dia)
dt = datetime(2021, 3, 1)
start_time = dt.replace(tzinfo=timezone.utc).timestamp()

Digite a data (DD-MM-AAAA): Todos os Tickets criados depois dessa data e atualizado recentemente
Data:01-06-2021


In [6]:
def api_url_check(url):
    response_tickets = requests.get(url,
            auth = HTTPBasicAuth(user, passw))

    #assert response_tickets.status_code == 200, f'Conexão não realizada'
    
def consulta_api(url):
    # Salvando resultado da consulta dos 1k tickets começando em 2021
    data_tickets = requests.get(url,
                auth = HTTPBasicAuth(user, passw)).text
    return data_tickets

def total_registros_retornados(api_retorno, list_deseralizado_tickets):
#     Conta quantos tickets foram retornados
    total = 0
    for itens in list_deseralizado_tickets:
        total += len(itens[str(api_retorno)])
        
    return total

def comparar_data_do_evento(data_evento, data_pesquisada):
  """
  Retorna se a data do evento é maior ou igual '=>' a data de pesquisa
  """
  return datetime.strptime(datetime.strftime(datetime.strptime(data_evento, '%Y-%m-%dT%H:%M:%S%z'), '%Y %m %d'), '%Y %m %d') >= data_pesquisada

In [7]:
day, month, year = map(int, str_tipo_consulta.split('-'))
dt = (datetime(year, month, day))

In [8]:
def consulta_tickets_atualizados_paginado(api_ticket_url):
  list_deseralizado_ids = []
    
  while True:

      print(api_ticket_url)
      api_url_check(api_ticket_url) # Testa url da API
      data_tickets = consulta_api(api_ticket_url) # Consulta os dados da API

      # Transformando o retorno em JSON
      deseralizado = json.loads(data_tickets)

      # Pega os ids e data de alteração do ticket
      list_ticket_alterados_pre_format = [[ticket['ticket_id'], ticket['created_at']] for ticket in deseralizado['audits']]

      if any(pd.DataFrame(list_ticket_alterados_pre_format)[1].astype('datetime64[ns]') > dt):
        list_tickets_id_alterados = [ticket[0] for ticket in list_ticket_alterados_pre_format if comparar_data_do_evento(ticket[1], dt)]

        # Separa os IDs q aparecem
        list_tickets_alterados_id = np.unique(list_tickets_id_alterados)

        list_deseralizado_ids.append(list_tickets_alterados_id)

        api_ticket_url = deseralizado['before_url']
        

      else:
          break

  return list_deseralizado_ids

In [9]:
# Fazendo um get request
response_audit = requests.get(f'{base_url}/api/v2/ticket_audits',
            auth = HTTPBasicAuth(user, passw))

assert response_audit.status_code == 200, f'Conexão não realizada'

In [10]:
ticket_ids = consulta_tickets_atualizados_paginado(f'{base_url}/api/v2/ticket_audits')

https://eslcloud.zendesk.com/api/v2/ticket_audits
https://eslcloud.zendesk.com/api/v2/ticket_audits.json?cursor=fDE2MjM2OTg0MTAuMHx8MTYwNDQzMjQyNjQ3Mg%3D%3D
https://eslcloud.zendesk.com/api/v2/ticket_audits.json?cursor=fDE2MjI4MjA5MjQuMHx8MTU4OTc1OTYxNjYxMQ%3D%3D
https://eslcloud.zendesk.com/api/v2/ticket_audits.json?cursor=fDE2MjE5NDYxNDIuMHx8MTU3NDgwNzA4MzYzMg%3D%3D


In [11]:
list_tickets_alterados_id = [item for sublist in ticket_ids for item in sublist]
list_tickets_alterados_id = np.unique(list_tickets_alterados_id)

## Testando URL para consulta da API de comentários

In [12]:
# Fazendo um get request
response = requests.get(f'{base_url}/api/v2/tickets/10476/comments',
            auth = HTTPBasicAuth(user, passw))

assert response.status_code == 200, f'Conexão não realizada'

In [13]:
# Salvando resultado da consulta
data = requests.get(f'{base_url}/api/v2/tickets/10476/comments',
            auth = HTTPBasicAuth(user, passw)).text

In [14]:
# Transformando o retorno em JSON
deseralizado = json.loads(data)

## Teste Localizar string nos cometários 

In [15]:
# Pegando os comentários internos
coments_interns = []
for coment in deseralizado['comments']:
    if coment['public'] == False:
        coments_interns.append(coment['body'])

assert len(coments_interns[0]) == 523, f'Informação faltando no texto do cometário'

# Regex para encontar pedaço de texto

assert re.search(r'\bFretes do frete\b',coments_interns[0]).group(0) == 'Fretes do frete', f'Regex não funcionou corretamente'

## Consulta de tickets 

In [16]:
def agrupa_valores_paginados(list_deseralizado_tickets, api_retorno):
    # Montando uma lista com todos os tickets consultados
    deseralizado_tickets = []

    for ticket_temp in list_deseralizado_tickets:
        deseralizado_tickets += ticket_temp[str(api_retorno)]

    assert len(deseralizado_tickets) == total_registros_retornados(api_retorno, list_deseralizado_tickets), f'Total de tickets nao confere com o consultado'
    
    return deseralizado_tickets

In [17]:
def consulta_api_paginada(api_ticket_url, consult_type = 'next_page'):
    list_deseralizado_tickets = []
    
    while True:

        api_url_check(api_ticket_url) # Testa url da API
        data_tickets = consulta_api(api_ticket_url) # Consulta os dados da API

        # Transformando o retorno em JSON
        deseralizado_tickets = json.loads(data_tickets)

        # Adiciona o retorno a uma lista
        list_deseralizado_tickets.append(deseralizado_tickets)

        try:
          if deseralizado_tickets[consult_type] == None:
              break
          else:
              api_ticket_url = deseralizado_tickets[consult_type]
        except:
            break
    
#     assert list_deseralizado_tickets[1]['results'][0]['id'] != list_deseralizado_tickets[0]['results'][0]['id'], f'Api não paginou corretamente'
    
    return list_deseralizado_tickets

In [18]:
string_tickets_id = str(list(list_tickets_alterados_id)).split('[')[1].split(']')[0]

In [19]:
def list_to_string_list(list_ints):
  strings_ids = str(list(list_ints)).split('[')[1].split(']')[0]

  return strings_ids

In [20]:
def consulta_lista_tickets(string_tickets_id):
  '''
  string_tickets_id = (List string)
  Retorna todos os tickets que possuem o ID na lista informada.
  '''
  api_ticket_url = f'{base_url}/api/v2/tickets/show_many?ids={string_tickets_id}'  

  return agrupa_valores_paginados(consulta_api_paginada(api_ticket_url, 'after_url'), 'tickets')

In [21]:
pbar = tqdm(range(int(len(list_tickets_alterados_id)/100)+1))
# del(deseralizado_tickets)

for idx_btach in pbar:  
  pbar.set_description("Rodadas de consultas de Tickets")
  idx_100 = idx_btach * 100

  string_tickets_id_batch = list_to_string_list(list_tickets_alterados_id[idx_100: idx_100+100])

  try: # Try para verificar se é a primeira interação ou se ja existem tickets consultados
    deseralizado_tickets
  except NameError:
      deseralizado_tickets = consulta_lista_tickets(string_tickets_id_batch)
  else:
      deseralizado_tickets = deseralizado_tickets + consulta_lista_tickets(string_tickets_id_batch)

Rodadas de consultas de Tickets: 100%|███████████████████████████████████████████████████| 6/6 [00:12<00:00,  2.12s/it]


In [22]:
assert len(deseralizado_tickets) == len(list_tickets_alterados_id), 'Não foram consultado todos os Tickets com alterações'

In [23]:
# campos da API com informação para o relatório por evento
relatorios_keys = ['id', 'subject','created_at','updated_at','requester_id','organization_id','status','group_id']

In [24]:
# campos da API com informação para o relatório por auditoria
relatorios_keys_audit = ['id', 'subject','created_at','updated_at','author_id','organization_id','status','group_id']

In [25]:
def format_date(date_to_parse):
    return parse(date_to_parse).date()

In [26]:
def relatorio_keys(tickets_consultados):
    # Retorna os campos do relatório para cada ticket consultado na API
    return [tickets_consultados.get(key) for key in relatorios_keys]

In [27]:
def analista_dos_tickets(deseralizado_tickets):
    """
    
    """
    
    analistas = []
#     for des_ticket in deseralizado_tickets['results']:
    for des_ticket in deseralizado_tickets:
      try:
        for field in des_ticket['custom_fields']:
            if field['id'] == 360020894432:
                analistas.append(field['value'])
      except:
        analistas.append(None)
    return analistas

In [28]:
def modulo_dos_tickets(deseralizado_tickets):
    modulo = []
#     for des_ticket in deseralizado_tickets['results']:
    for des_ticket in deseralizado_tickets:
        for field in des_ticket['custom_fields']:
            if field['id'] == 360020903331:
                modulo.append(field['value'])
    return modulo

In [29]:
def motivo_contato_dos_tickets(deseralizado_tickets):
    motivo = []
    
    for des_ticket in deseralizado_tickets:
      try:
        for field in des_ticket['custom_fields']:
            if field['id'] == 360040996232:
                motivo.append(field['value'])
      except:
        motivo.append(None)
    return motivo

In [30]:
# Lista com os as informaçõs dos tickets tickets
# ticket_list = [relatorio_keys(ticket_temp) for ticket_temp in deseralizado_tickets['results']]
ticket_list = [relatorio_keys(ticket_temp) for ticket_temp in deseralizado_tickets]

In [31]:
tickets = pd.DataFrame(ticket_list,
                       columns=['id', 'assunto','criacao','atualizado','id_solicitante','id_organizacao','status','id_status'])

In [32]:
tickets['criacao'] = [parse(date) for date in tickets['criacao']]
tickets['atualizado'] = [parse(date) for date in tickets['atualizado']]

In [33]:
tickets['hora_criacao'] = tickets.apply(lambda x: x['criacao'].hour, axis=1)
tickets['minuto_criacao'] = tickets.apply(lambda x: x['criacao'].minute, axis=1)

tickets['hora_finalizacao'] = tickets.apply(lambda x: x['criacao'].hour, axis=1)
tickets['minuto_finalizacao'] = tickets.apply(lambda x: x['criacao'].minute, axis=1)

In [34]:
assert len(tickets['id_solicitante']) == len(deseralizado_tickets), 'Quantidade de tickets com diferença no id_solicitante'

In [35]:
# Formatando a data para dd/mm/aaaa
tickets['criacao'] = tickets['criacao'].apply(lambda x: x.strftime("%d/%m/%Y"))
tickets['atualizado'] = tickets['atualizado'].apply(lambda x: x.strftime("%d/%m/%Y"))

In [36]:
tickets['analista'] = analista_dos_tickets(deseralizado_tickets)
# tickets['modulo'] = modulo_dos_tickets(deseralizado_tickets)
tickets['motivo_contato'] = motivo_contato_dos_tickets(deseralizado_tickets)

In [37]:
# parse(tickets['criacao'][0]).strftime("%d %m %Y")

# Remove os itens duplicado das duas consultas
tickets = tickets.drop_duplicates()

## Comentários dos tickets consultados

In [38]:
ids_para_consulta = tickets.id.values

In [39]:
data = requests.get(f'{base_url}/api/v2/tickets/{str(11171)}/comments',
        auth = HTTPBasicAuth(user, passw)).text

# Transformando o retorno em JSON
deseralizado_tickets_comentario = json.loads(data)

assert deseralizado_tickets_comentario['comments'][0]['id'] == 1546957011772, f'Cometários estão sendo trazidos errado'

In [40]:
# Recebe o retorno da API e devolve uma lista com ID, cometário, e data de criação
def comentario_interno(deseralizado, ticket_id):
    # Pegando os comentários internos
    coments_interns = []
    for coment in deseralizado['comments']:
#         if coment['public'] == False:
        coments_interns.append([ticket_id, coment['id'], coment['body'], coment['created_at'], coment['public']])
            
    return coments_interns

In [41]:
# Consulta os comentários de um ticket
def comentarios_do_ticket(ticket_id):
    
    # Salvando resultado da consulta
    data = requests.get(f'{base_url}/api/v2/tickets/{str(ticket_id)}/comments',
            auth = HTTPBasicAuth(user, passw)).text
    
    # Transformando o retorno em JSON
    deseralizado = json.loads(data)
    
    # Pegando comentários internos
    coments_interns = comentario_interno(deseralizado, ticket_id)
    
    # retorna os comentários do ticket
    return coments_interns

In [42]:
# Consulta os comentários do tickets
list_ticket_comentarios = []

# for idx in tqdm(ids_para_consulta[-10:]):
for idx in tqdm(ids_para_consulta):
    list_ticket_comentarios.append(comentarios_do_ticket(idx))

100%|████████████████████████████████████████████████████████████████████████████████| 595/595 [05:55<00:00,  1.67it/s]


In [43]:
# Transforma os comentários em uma lista
list_all_coments = []

for ticket_values in list_ticket_comentarios: # Uma vez para cada ticket
    for coment in ticket_values: # Uma vez para cada comentário no ticket
        list_all_coments.append(coment)

In [44]:
# Cria DF com os comentários obtidos dos Tickets
df_ticket_comentarios = pd.DataFrame(list_all_coments, columns=['id', 'id_comentario', 
                                                                'texto_comentario','data_comentario','publico'])

In [45]:
# Convertando as datas do dataframe para date
df_ticket_comentarios['data_comentario'] = [parse(date) for date in df_ticket_comentarios['data_comentario']]

# Criando coluna com dia mes e ano separados
df_ticket_comentarios['data_comentario_dia'] = df_ticket_comentarios.apply(lambda x: x['data_comentario'].day, axis=1)
df_ticket_comentarios['data_comentario_mes'] = df_ticket_comentarios.apply(lambda x: x['data_comentario'].month, axis=1)
df_ticket_comentarios['data_comentario_ano'] = df_ticket_comentarios.apply(lambda x: x['data_comentario'].year, axis=1)

# Formatando a data para dd/mm/aaaa
df_ticket_comentarios['data_comentario'] = df_ticket_comentarios['data_comentario'].apply(lambda x: x.strftime("%d/%m/%Y"))

df_ticket_comentarios.head()

,id,id_comentario,texto_comentario,data_comentario,publico,data_comentario_dia,data_comentario_mes,data_comentario_ano
0,9642,1441245577212,na consulta do movimento não sai a sigla da UF...,01/03/2021,True,1,3,2021
1,9642,1440071950031,\n\n\n\n![](https://eslcloud.zendesk.com/attac...,01/03/2021,True,1,3,2021
2,9642,1442207272512,"Prezado Sivanildo, bom dia!\n\nSua solicitação...",02/03/2021,True,2,3,2021
3,9642,1440974366311,No detalhes do frete não está apresentando a U...,02/03/2021,False,2,3,2021
4,9642,1479211510871,![](https://eslcloud.zendesk.com/attachments/...,25/03/2021,False,25,3,2021


## Juntando comentários com as informações dos tickets

In [46]:
df_ticket_com_comentários = pd.merge(df_ticket_comentarios, tickets, on='id')

In [47]:
mask = np.isin(tickets.id.unique(), df_ticket_comentarios.id.unique())
assert (len(np.unique(mask)) == 1) and (np.unique(mask) == True)[0], 'Existem tickets que não estão na lista dos comentários'

## Organizações 

In [48]:
# URL para todos os tickets nao fechados
api_organizaot_url = f'{base_url}/api/v2/organizations'

organizacao_deseralizada = agrupa_valores_paginados(consulta_api_paginada(api_organizaot_url),'organizations')

In [49]:
list_organizacoes = []
for organiza in organizacao_deseralizada:
    list_organizacoes.append([organiza['id'], organiza['name']])

In [50]:
# Cria dataframe com as organizações
df_organizacoes = pd.DataFrame(list_organizacoes,columns=['id_organizacao','nome_organizacao'])

In [51]:
# pd.merge(df_ticket_com_comentários, df_organizacoes, on='id_organizacao').id.nunique()
assert df_ticket_com_comentários.set_index('id_organizacao').join(df_organizacoes.set_index('id_organizacao'), on='id_organizacao', how='left').index.size == df_ticket_com_comentários.index.size, 'Quantidade de tickets alterada após incluir as organizações'

In [52]:
df_ticket_com_comentários = pd.merge(df_ticket_com_comentários, df_organizacoes, on='id_organizacao',how='left')
# df_ticket_com_comentários = df_ticket_com_comentários.set_index('id_organizacao').join(df_organizacoes.set_index('id_organizacao'), on='id_organizacao', how='left')

## Solicitante 

In [53]:
from nltk import flatten

In [54]:
# IDs dos usuarios dos tickets
id_solicitantes = df_ticket_com_comentários['id_solicitante'].unique()

In [55]:
list_todos_usuarios = []

for ticket_batch in range(int(len(list(df_ticket_com_comentários['id_solicitante'].unique()))/100) +1):
  count = ticket_batch*100 

  api_usuarios = (requests.get(f'{base_url}/api/v2/users/show_many.json?ids={list(id_solicitantes[count: count+100])}',
          auth = HTTPBasicAuth(user, passw)).text)

  list_todos_usuarios.append(json.loads(api_usuarios)['users'])
  # usuarios_deseralizado = json.loads(api_usuarios)['users']
  

In [56]:
usuarios_deseralizado = list(flatten(list_todos_usuarios))

In [57]:
list_usuarios = []

for user in usuarios_deseralizado:
    list_usuarios.append([user['id'], user['name']])

In [58]:
# Cria dataframe com os usuarios
df_solicitantes = pd.DataFrame(list_usuarios,columns=['id_solicitante','nome_usuario'])

In [59]:
assert pd.merge(df_ticket_com_comentários, df_solicitantes, on='id_solicitante', how='left').index.size == df_ticket_com_comentários.index.size, 'Solicitante está removendo tickets da lista'

In [60]:
# Megre usuarios com os tickets
# df_ticket_com_comentários = pd.merge(df_ticket_com_comentários, df_solicitantes, on='id_solicitante')
df_ticket_com_comentários = pd.merge(df_ticket_com_comentários, df_solicitantes, on='id_solicitante', how='left')

## Tempo de cada analista 

In [61]:
dict_agente_analistas = {'Marcelo':'mlourenco',
'matheus_migliato':'mmigliato',
'caroline_vantim':'cvantim',
'bruno_ricardo':'bhernandes',
'lucas_freitas':'lfreitas',
'Fernando':'fluiz',
'joão_guilherme':'cvantim'}

In [62]:
def tempo_analista(row):
    
    if not row['publico']:
        try:
            analista_tempo = row['texto_comentario'].split('@@')[1].split(',') # Pega o analista e o tempo em uma lista
            analista = analista_tempo[0] # Obtem o nome do analista
            tempo = re.search(r'\d+', analista_tempo[1]).group() # Obtem o tempo informado (ignorando o texto)
            
            # return [row['id'], analista, tempo]
        except:
            analista = None
            tempo = None
            
        return [row['id_comentario'], analista, tempo]

In [63]:
# Lista com o id do ticket, nome do analista e tempo gasto
tempo_comentarios_internos = df_ticket_comentarios.apply(lambda x: tempo_analista(x),axis=1)

In [64]:
# Lista de cada comentário informando tempo
list_tickets_comentarios = [analista_tempo for analista_tempo in tempo_comentarios_internos if analista_tempo is not None]

In [65]:
# Criando df com os dados do ticket e o tempo e analista
df_tempo_comentario_analista = pd.DataFrame(list_tickets_comentarios, columns=['id_comentario',
                                                                               'analista_comentario','tempo_comentario'])

In [66]:
assert pd.merge(df_ticket_com_comentários, df_tempo_comentario_analista, on='id_comentario', how='left').index.size == df_ticket_com_comentários.index.size, 'Comentários estão removendo tickets da consulta'

In [67]:
# Juntando tempo com o comentário
df_ticket_com_comentários = pd.merge(df_ticket_com_comentários, df_tempo_comentario_analista, on='id_comentario')

In [70]:
# Salvando dados
filename = 'consultas/'
os.makedirs(os.path.dirname(filename), exist_ok=True)

df_ticket_com_comentários.drop(['id_comentario', 'publico', 'id_solicitante', 'id_organizacao', 'id_status'],1).to_csv(f'{filename}/tickets_{datetime.now().date().day}_{datetime.now().date().month}_{datetime.now().date().year}_{datetime.now().hour}_{datetime.now().minute}_{datetime.now().second}.csv', sep = ';')